<a href="https://colab.research.google.com/github/vivianusdjpy/SUPERVISED-LEARNING-WITH-EDA/blob/master/Supervised_Learning_with_Exploratory_Data_Analysis_Mchezo_Pesa_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Learning with Exploratory Data Analysis:
# Mchezo Pesa Project

## 1. Defining the Question

You have been recruited as a football analyst in a company - Mchezopesa Ltd and tasked to accomplish the task below.

A prediction result of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly (include rank in your training).

### a) Specifying the Question


Mchezo pesa is a football analyst company in East Africa. Mchezo-Pesa would like to predict the result of a game based whether the teams playing are in their home country or not and whether the game is friendly or not.



### b) Defining the Metric for Success
This project will be successful when:
1)We accurately predict the winning team based on the two factors:

       a)Home team or away team

       b)The type of tournament 

2)The model achieves atleast 80% accuracy

3)Have the lowest RMSE score possible


### c) Understanding the context 
The Coca-Cola ranking system was designed by FIFA and named 'SUM'.
This ranking method utilises a metric that adds or subtracts points to the previous score instead of finding the average score.
The lowest score which may be negative indicates the worst team while the highest score indicated the best team.

### d)Experimental Design
The project was undertaken using the following design
Datasets(Fifa-ranking.csv + results.csv)

Merging datasets

Performing EDA

Feature Engineering

Polynomial Regression

Logistic regression

Computing the RMSE

Plotting residual plots

Assess their heteroscedasticity using Barlette's test

Challenging the solution

Conclusion

### e) Data Relevance
The relavance of our data will be answered by the following questions.

How accurate is the data at predicting the outcome of the games?

Was the dataset sufficient?

Was the data biased?

Is the data source a reliable source?

## 2. Reading the Data

In [0]:
#loading the libraries
import pandas as pd
import numpy as np
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')


In [50]:
#loading the FiFa ranking dataset while specifying the datetime column
ranking = pd.read_csv('/content/fifa_ranking.csv', parse_dates=['rank_date'])
ranking.head(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [51]:
# Loading the FiFa results Dataset while specifying the datetime column
results = pd.read_csv('/content/results.csv',parse_dates=['date'])
results.head(5)
#

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


## 3. Cleaning the Data

FIFA Ranking dataset

In [52]:
# Determining the no. of records in our dataset and the 
ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57793 entries, 0 to 57792
Data columns (total 16 columns):
rank                       57793 non-null int64
country_full               57793 non-null object
country_abrv               57793 non-null object
total_points               57793 non-null float64
previous_points            57793 non-null int64
rank_change                57793 non-null int64
cur_year_avg               57793 non-null float64
cur_year_avg_weighted      57793 non-null float64
last_year_avg              57793 non-null float64
last_year_avg_weighted     57793 non-null float64
two_year_ago_avg           57793 non-null float64
two_year_ago_weighted      57793 non-null float64
three_year_ago_avg         57793 non-null float64
three_year_ago_weighted    57793 non-null float64
confederation              57793 non-null object
rank_date                  57793 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(9), int64(3), object(3)
memory usage: 7.1+ MB


In [53]:
#checking for duplicates in the results column
ranking.duplicated().sum()

37

In [54]:
#dropping the duplicated columns in the ranking dataset
#keeping the first record of the duplicates
ranking.drop_duplicates(subset = None, keep = 'first', inplace = True)
ranking.shape

(57756, 16)

In [55]:
# Previewing the top of our dataset
ranking.head(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08


In [56]:
# Previewing the bottom of our dataset
ranking.tail(5)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
57788,206,Anguilla,AIA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57789,206,Bahamas,BAH,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONCACAF,2018-06-07
57790,206,Eritrea,ERI,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57791,206,Somalia,SOM,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,2018-06-07
57792,206,Tonga,TGA,0.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OFC,2018-06-07


In [57]:
# Checking the properties of the ranking dataset
ranking.describe()

,rank,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted
count,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000,57756.000000
mean,101.618118,121.971151,332.339965,-0.009938,61.751362,61.751362,60.953507,30.476830,59.734768,17.920469,59.114466,11.822922
std,58.635096,260.479191,302.964176,5.802788,138.042006,138.042006,137.712945,68.856514,136.319119,40.895761,135.540135,27.108034
min,1.000000,0.000000,0.000000,-72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,51.000000,0.000000,56.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,101.000000,0.000000,272.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,152.000000,91.500000,525.000000,1.000000,30.447500,30.447500,26.400000,13.200000,21.250000,6.380000,20.795000,4.162500
max,209.000000,1775.030000,1920.000000,92.000000,1158.660000,1158.660000,1169.570000,584.790000,1159.710000,347.910000,1200.770000,240.150000


In [58]:
#chosing the relevant columns to work with.
ranking_1 = ranking[["rank","country_full","confederation","rank_date"]]
ranking_1


,rank,country_full,confederation,rank_date
0,1,Germany,UEFA,1993-08-08
1,2,Italy,UEFA,1993-08-08
2,3,Switzerland,UEFA,1993-08-08
3,4,Sweden,UEFA,1993-08-08
4,5,Argentina,CONMEBOL,1993-08-08
...,...,...,...,...
57788,206,Anguilla,CONCACAF,2018-06-07
57789,206,Bahamas,CONCACAF,2018-06-07
57790,206,Eritrea,CAF,2018-06-07
57791,206,Somalia,CAF,2018-06-07


In [59]:
#creating a copy of ranking_1 dataset
ranking_2 = ranking_1
# Separate the date column into Year and Month of the fifa_ranking dataset
ranking_2['Year'] = ranking_1.rank_date.dt.year
ranking_2['month'] =ranking_1.rank_date.dt.month
ranking_2.head()

,rank,country_full,confederation,rank_date,Year,month
0,1,Germany,UEFA,1993-08-08,1993,8
1,2,Italy,UEFA,1993-08-08,1993,8
2,3,Switzerland,UEFA,1993-08-08,1993,8
3,4,Sweden,UEFA,1993-08-08,1993,8
4,5,Argentina,CONMEBOL,1993-08-08,1993,8


In [69]:
#creating a copy of the ranking_2 dataframe
ranking_3 =  ranking_2
#renaming the country_full column to home_team
ranking_3 = ranking_3.rename({'country_full': 'home_team'}, axis = 1)
ranking_3.columns

Index(['rank', 'home_team', 'confederation', 'rank_date', 'Year', 'month'], dtype='object')

# Fifa Results Dataset

In [60]:
results.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [61]:
#choosing the relevant columns in the results dataset
results = results[["date","home_team","away_team","home_score","away_score","tournament","country","neutral"]]
results

,date,home_team,away_team,home_score,away_score,tournament,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Scotland,False
...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Samoa,True
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Samoa,True
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Egypt,True
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,India,True


In [62]:
#checking for duplicates in the ranking column
results.duplicated().sum()

0

In [63]:
results['date'].describe()

count                   40839
unique                  15115
top       2012-02-29 00:00:00
freq                       66
first     1872-11-30 00:00:00
last      2019-07-20 00:00:00
Name: date, dtype: object

In [64]:
#dropping the row entries before 1993 since they do not have corresponding records 
#in the ranking  dataframe.
results_1 = results
results_1 = results[results["date"] > '1993-01-01']
results_1.shape
#dropped 40839-23477 = 17362 row entries

(23477, 8)

In [65]:
#creating a copy of results_1 dataset
results_2 = results_1
# Separate the date column into Year and Month of the fifa_ranking dataset
results_2['Year'] = results_1.date.dt.year
results_2['month'] =results_1.date.dt.month
results_2.head()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral,Year,month
17362,1993-01-02,Gabon,Burkina Faso,1,1,Friendly,Gabon,False,1993,1
17363,1993-01-02,Kuwait,Lebanon,2,0,Friendly,Kuwait,False,1993,1
17364,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Gabon,True,1993,1
17365,1993-01-03,Gabon,Ghana,2,3,Friendly,Gabon,False,1993,1
17366,1993-01-08,Uganda,Tanzania,3,1,Friendly,Uganda,False,1993,1


# Merging the Two datasets

In [74]:
## Merging the two datasets using the similar columns : home_team, year and month
fifa_df = pd.merge(results_2, ranking_3, how='left', on = ['home_team', 'Year', 'month'])
fifa_df.head()

,date,home_team,away_team,home_score,away_score,tournament,country,neutral,Year,month,rank,confederation,rank_date
0,1993-01-02,Gabon,Burkina Faso,1,1,Friendly,Gabon,False,1993,1,NaN,NaN,NaT
1,1993-01-02,Kuwait,Lebanon,2,0,Friendly,Kuwait,False,1993,1,NaN,NaN,NaT
2,1993-01-03,Burkina Faso,Mali,1,0,Friendly,Gabon,True,1993,1,NaN,NaN,NaT
3,1993-01-03,Gabon,Ghana,2,3,Friendly,Gabon,False,1993,1,NaN,NaN,NaT
4,1993-01-08,Uganda,Tanzania,3,1,Friendly,Uganda,False,1993,1,NaN,NaN,NaT


In [73]:
#obtaining information about the merged dataset
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23530 entries, 0 to 23529
Data columns (total 13 columns):
date             23530 non-null datetime64[ns]
home_team        23530 non-null object
away_team        23530 non-null object
home_score       23530 non-null int64
away_score       23530 non-null int64
tournament       23530 non-null object
country          23530 non-null object
neutral          23530 non-null bool
Year             23530 non-null int64
month            23530 non-null int64
rank             18593 non-null float64
confederation    18593 non-null object
rank_date        18593 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(1), int64(4), object(5)
memory usage: 2.4+ MB


## 5. Tidying the Dataset

In [0]:
# Checking for Outliers
#

In [0]:
# Checking for Anomalies
#

In [0]:
# Identifying the Missing Data
#

In [0]:
# Dealing with the Missing Data
#

In [0]:
# More data cleaning procedures
#

## 6. Exploratory Analysis

In [0]:
# Ploting the univariate summaries and recording our observations
#

## 7. Implementing the Solution

In [0]:
# Implementing the Solution
# 

## 8. Challenging the solution

> The easy solution is nice because it is, well, easy, but you should never allow those results to hold the day. You should always be thinking of ways to challenge the results, especially if those results comport with your prior expectation.






In [0]:
# Reviewing the Solution 
#

## 9. Follow up questions

> At this point, we can refine our question or collect new data, all in an iterative process to get at the truth.



### a). Did we have the right data?

### b). Do we need other data to answer our question?

### c). Did we have the right question?